# Radiomics Feature Extraction

Extracting features from PET and CT images.

In [1]:
import os
import nrrd
import utils 
import radiomics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from feature_extraction import feature_extractor

%matplotlib inline

In [128]:
path_masksdir = './../../data_source/images/masks_removed_broken_images/'
path_ct_imagedir = './../../data_source/images/ct_removed_broken_images/'
path_pet_imagedir = './../../data_source/images/pet_removed_broken_images//'
#path_masksdir = './../../data_source/images/masks_nrrd/'
#path_ct_imagedir = './../../data_source/images/ct_nrrd/'
#path_pet_imagedir = './../../data_source/images/pet_nrrd/'

In [129]:
paths_ct_images = utils.sample_paths(
    path_ct_imagedir, path_masksdir, target_format='nrrd'
)
paths_pet_images = utils.sample_paths(
    path_pet_imagedir, path_masksdir, target_format='nrrd'
)

In [130]:
# Sanity check.
len(paths_ct_images), len(paths_pet_images)

(195, 195)

## Setup

In [131]:
# Texture features are shown to be more stable after GL normalizations.
# Ref: Voxel size and gray level normalization of CT radiomic features in lung cancer.
def z_scoring(image):
    
    _image = np.copy(image)
    image_z_scored = (_image - np.mean(_image)) / (np.std(_image) + 1e-10)

    return image_z_scored


def bin_widths(path_images, nbins, n=3, z_scoring=False):
    """Average min and max to compare histogram across stacks.
    Fixed bin iwdth (and not fixed bin size) to compare texture
    features across stacks.
    
    """
    img_max, img_min = [], []
    for image_path in path_images:
        
        image, _ = nrrd.read(image_path['Image'])
        mask, _ = nrrd.read(image_path['Mask'])
        
        _cropped_image = image * mask
        # NOTE: Want min that is not background. Returned 
        # cropped image changes shape.
        cropped_image = _cropped_image[_cropped_image != 0]
        
        if z_scoring:
            cropped_image = z_score_transform(cropped_image)
        
        img_max.append(np.ceil(np.max(cropped_image)))
        img_min.append(np.floor(np.min(cropped_image)))
  
    return (np.mean(img_max) - np.mean(img_min)) / nbins

In [132]:
# NOTE: 
# * Calc GL discr from tumor region only to best preserve tumor texture.
# * Slice removal requires different GL bins widths.

print(bin_widths(paths_pet_images, 32))
print(bin_widths(paths_pet_images, 64)) 
print(bin_widths(paths_pet_images, 128))

0.4043269230769231
0.20216346153846154
0.10108173076923077


In [133]:
# TODO:
# * Calc GL discr from tumor region only to best preserve tumor texture.
# * Slice removal requires different GL bins widths.

print(bin_widths(paths_ct_images, 32))
print(bin_widths(paths_ct_images, 64)) 
print(bin_widths(paths_ct_images, 128))

40.611378205128204
20.305689102564102
10.152844551282051


In [135]:
# TODO: 
# * Calc GL discr from tumor region only to best preserve tumor texture.
# * Slice removal requires different GL bins widths.

print(bin_widths(paths_ct_images, 32, z_scoring=True))
print(bin_widths(paths_ct_images, 64, z_scoring=True))
print(bin_widths(paths_ct_images, 128, z_scoring=True))

0.589102564102564
0.294551282051282
0.147275641025641


## Calculate features

In [143]:
ct_param_files = [
    #'./parameter_files/shape/shape_config.yaml',
    
    #'./parameter_files/firstorder_original_images/ct32_firstorder_original_images_config.yaml',
    #'./parameter_files/firstorder_original_images/ct64_firstorder_original_images_config.yaml',
    #'./parameter_files/firstorder_original_images/ct128_firstorder_original_images_config.yaml'
    #'./parameter_files/texture_original_images/ct32_texture_original_images_config.yaml',
    #'./parameter_files/texture_original_images/ct64_texture_original_images_config.yaml',
    #'./parameter_files/texture_original_images/ct128_texture_original_images_config.yaml',
    
    './parameter_files/firstorder_removed_broken_images/ct32_firstorder_removed_broken_images_config.yaml',
    './parameter_files/firstorder_removed_broken_images/ct64_firstorder_removed_broken_images_config.yaml',
    './parameter_files/firstorder_removed_broken_images/ct128_firstorder_removed_broken_images_config.yaml',
    #'./parameter_files/texture_removed_broken_images/ct32_texture_removed_broken_images_config.yaml',
    #'./parameter_files/texture_removed_broken_images/ct64_texture_removed_broken_images_config.yaml',
    #'./parameter_files/texture_removed_broken_images/ct128_texture_removed_broken_images_config.yaml',
]
ct_feature_files = [
    #'./../../data_source/radiomic_features/shape/shape_original_masks.csv',
    #'./../../data_source/radiomic_features/shape/shape_removed_broken_images.csv',

    #'./../../data_source/radiomic_features/firstorder_original_images/ct32_firstorder_original_images.csv',
    #'./../../data_source/radiomic_features/firstorder_original_images/ct64_firstorder_original_images.csv',
    #'./../../data_source/radiomic_features/firstorder_original_images/ct128_firstorder_original_images.csv'
    #'./../../data_source/radiomic_features/texture_original_images/ct32_texture_original_images.csv',
    #'./../../data_source/radiomic_features/texture_original_images/ct64_texture_original_images.csv',
    #'./../../data_source/radiomic_features/texture_original_images/ct128_texture_original_images.csv',
    
    './../../data_source/radiomic_features/firstorder_removed_broken_images/ct32_firstorder_removed_broken_images_config.csv',
    './../../data_source/radiomic_features/firstorder_removed_broken_images/ct64_firstorder_removed_broken_images_config.csv',
    './../../data_source/radiomic_features/firstorder_removed_broken_images/ct128_firstorder_removed_broken_images_config.csv',
    #'./../../data_source/radiomic_features/texture_removed_broken_images/ct32_texture_removed_broken_images_config.csv',
    #'./../../data_source/radiomic_features/texture_removed_broken_images/ct64_texture_removed_broken_images_config.csv',
    #'./../../data_source/radiomic_features/texture_removed_broken_images/ct128_texture_removed_broken_images_config.csv',
]
for ct_param_file, ct_feature_file in zip(ct_param_files, ct_feature_files):
    feature_extractor(
        param_file=ct_param_file, 
        paths_to_images_and_masks=paths_ct_images, 
        verbose=1, 
        path_to_results=ct_feature_file,
        n_jobs=None, 
        drop_missing=True, 
        variance_thresh=0.0
    )

Initiated feature extraction.


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    5.9s
[Parallel(n_jobs=3)]: Done 190 out of 195 | elapsed:   14.4s remaining:    0.4s
[Parallel(n_jobs=3)]: Done 195 out of 195 | elapsed:   14.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    5.7s
[Parallel(n_jobs=3)]: Done 190 out of 195 | elapsed:   13.7s remaining:    0.4s
[Parallel(n_jobs=3)]: Done 195 out of 195 | elapsed:   13.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    3.7s
[Parallel(n_jobs=3)]: Done 195 out of 195 | elapsed:   13.8s finished


In [142]:
pet_param_files = [
    #'./parameter_files/firstorder_original_images/pet32_firstorder_original_images_config.yaml',
    #'./parameter_files/firstorder_original_images/pet64_firstorder_original_images_config.yaml',
    #'./parameter_files/firstorder_original_images/pet128_firstorder_original_images_config.yaml'
    #'./parameter_files/texture_original_images/pet32_texture_original_images_config.yaml',
    #'./parameter_files/texture_original_images/pet64_texture_original_images_config.yaml',
    #'./parameter_files/texture_original_images/pet128_texture_original_images_config.yaml',
    
    #'./parameter_files/firstorder_removed_broken_images/pet32_firstorder_removed_broken_images_config.yaml',
    #'./parameter_files/firstorder_removed_broken_images/pet64_firstorder_removed_broken_images_config.yaml',
    #'./parameter_files/firstorder_removed_broken_images/pet128_firstorder_removed_broken_images_config.yaml',
    './parameter_files/texture_removed_broken_images/pet32_texture_removed_broken_images_config.yaml',
    './parameter_files/texture_removed_broken_images/pet64_texture_removed_broken_images_config.yaml',
    './parameter_files/texture_removed_broken_images/pet128_texture_removed_broken_images_config.yaml',
]
pet_feature_files = [
    #'./../../data_source/radiomic_features/firstorder_original_images/pet32_firstorder_original_images.csv',
    #'./../../data_source/radiomic_features/firstorder_original_images/pet64_firstorder_original_images.csv',
    #'./../../data_source/radiomic_features/firstorder_original_images/pet128_firstorder_original_images.csv'
    #'./../../data_source/radiomic_features/texture_original_images/pet32_texture_original_images.csv',
    #'./../../data_source/radiomic_features/texture_original_images/pet64_texture_original_images.csv',
    #'./../../data_source/radiomic_features/texture_original_images/pet128_texture_original_images.csv',
    
    #'./../../data_source/radiomic_features/firstorder_removed_broken_images/pet32_firstorder_removed_broken_images_config.csv',
    #'./../../data_source/radiomic_features/firstorder_removed_broken_images/pet64_firstorder_removed_broken_images_config.csv',
    #'./../../data_source/radiomic_features/firstorder_removed_broken_images/pet128_firstorder_removed_broken_images_config.csv',
    './../../data_source/radiomic_features/texture_removed_broken_images/pet32_texture_removed_broken_images_config.csv',
    './../../data_source/radiomic_features/texture_removed_broken_images/pet64_texture_removed_broken_images_config.csv',
    './../../data_source/radiomic_features/texture_removed_broken_images/pet128_texture_removed_broken_images_config.csv',
]
for pet_param_file, pet_feature_file in zip(pet_param_files, pet_feature_files):
    feature_extractor(
        param_file=pet_param_file, 
        paths_to_images_and_masks=paths_pet_images, 
        verbose=1, 
        path_to_results=pet_feature_file,
        n_jobs=None, 
        drop_missing=True, 
        variance_thresh=0.0
    )

Initiated feature extraction.


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    5.6s
[Parallel(n_jobs=3)]: Done 190 out of 195 | elapsed:   13.0s remaining:    0.3s
[Parallel(n_jobs=3)]: Done 195 out of 195 | elapsed:   13.2s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    4.9s
[Parallel(n_jobs=3)]: Done 190 out of 195 | elapsed:   12.8s remaining:    0.3s
[Parallel(n_jobs=3)]: Done 195 out of 195 | elapsed:   13.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  82 tasks      | elapsed:    6.0s
[Parallel(n_jobs=3)]: Done 190 out of 195 | elapsed:   13.1s remaining:    0.3s
[Parallel(n_jobs=3)]: Done 195 out of 195 | elapsed:   13.3s finished
